In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from IPython.display import Audio, display
import anthropic

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")
    
GPT_MODEL = "gpt-4o-mini"
openai = OpenAI()

ANTHROPIC_MODEL = "claude-3-haiku-20240307"
claude = anthropic.Anthropic()

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "When booking a flight always check if the desired time is really available."

translate_system_message = "You are a language assistant that translates text between English and German. "
translate_system_message += "Translate the given text from English to German accurately while preserving the meaning and tone. "
translate_system_message += "Return only the translated text without explanations."

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

flight_times = {"london": ["08:00", "11:42", "13:30", "15:47"], 
                "paris": ["07:30", "10:44", "17:30", "22:37"], 
                "tokyo": ["08:54", "14:59"], 
                "berlin": ["06:52", "09:43", "13:56"]}

def get_flight_times(destination_city):
    print(f"Tool get_flight_times called for {destination_city}")
    city = destination_city.lower()
    return flight_times.get(city, "Unknown")    

def book_flight(destination_city, time):
    print(f"Booking flight to {destination_city} at {time}")
    return f"Flight to {destination_city.title()} booked for {time}. Confirmation sent!"

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

flight_times_function = {
    "name": "get_flight_times",
    "description": "Get the times of fights to location. Call this whenever you need to know the flight times, for example when a customer want to book a flight to a city, provide him with the time options.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

book_flight_function = {
    "name": "book_flight",
    "description": "Book a flight to a city at a specific time.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "City to fly to"
            },
            "time": {
                "type": "string",
                "description": "Departure time in HH:MM"
            }
        },
        "required": ["destination_city", "time"]
    }
}

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": flight_times_function},
    {"type": "function", "function": book_flight_function}
]

def handle_tool_call(message):
    tool_responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Handling tool call for function: {function_name}, arguments: {arguments}")
        city = arguments.get("destination_city")

        if function_name == "get_ticket_price":
            result = get_ticket_price(city)
        elif function_name == "get_flight_times":
            result = get_flight_times(city)
        elif function_name == "book_flight":
            time = arguments.get("time")
            result = book_flight(city, time)
        else:
            result = "Unknown function"

        tool_responses.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": json.dumps(result)
        })
    return tool_responses, city

def translate_text(text, to_language="German"):
    """Translate text using Claude Haiku"""
    if not text:
        return ""
    
    response = claude.messages.create(
        model=ANTHROPIC_MODEL,
        max_tokens=1000,
        messages=[
            {"role": "system", "content": translate_system_message},
            {"role": "user", "content": f"Translate this text to {to_language}: {text}"}
        ]
    )
    
    return response.content[0].text

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

def chat(message, history):
    # Translate user message to English if needed (for processing)
    # For simplicity, we're assuming input is in English
    original_message = message
    
    messages = [{"role": "system", "content": system_message}]
    
    # Convert history to the format expected by OpenAI API
    for h in history:
        # Extract only the original message (first part before the translation)
        if h["role"] == "user" or h["role"] == "assistant":
            content = h["content"].split("\n\n**German:** ")[0]
            if h["role"] == "user":
                content = content.replace("**English:** ", "")
            messages.append({"role": h["role"], "content": content})
    
    # Add the current user message
    messages.append({"role": "user", "content": original_message})
    
    response = openai.chat.completions.create(model=GPT_MODEL, messages=messages, tools=tools)
    image = None

    if response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        
        tool_messages, city = handle_tool_call(assistant_message)

        # Append assistant tool call message and tool responses
        messages.append(assistant_message)
        messages.extend(tool_messages)
        
        # Send updated messages back to OpenAI
        response = openai.chat.completions.create(model=GPT_MODEL, messages=messages)

    reply = response.choices[0].message.content
    
    # Translate the reply to German
    german_reply = translate_text(reply)
    
    # Format the message to show both English and German
    formatted_reply = f"{reply}\n\n**German:** {german_reply}"
    
    # Add to history with both original and translated text
    history += [{"role": "assistant", "content": formatted_reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now
    talker(reply)
    
    return history

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        # Translate user message to German
        german_translation = translate_text(message)
        
        # Format the message to show both English and German
        formatted_message = f"**English:** {message}\n\n**German:** {german_translation}"
        
        history += [{"role": "user", "content": formatted_message}]
        return "", history
        
    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
       chat, inputs=[entry, chatbot], outputs=[chatbot]
    )

    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Startklar\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Startklar\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Startklar\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Startklar\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Startklar\anaconda3\envs\llms\Lib\site-packages\anyio\to_thread.py", line 56, in run_